In [1]:
# You only need to run this cell once (per time you launch the notebook, in colab or jupyterlab)
# Once you have installed gymnasium, you can comment out the next line
!pip install gymnasium
!pip install "gymnasium[classic-control]" # use quotes on mac
# !pip install "gymnasium[box2d]" # has known installation problems

^C


ERROR: Invalid requirement: '#'


In [1]:
import gymnasium as gym
import random
import time
import numpy as np

In [2]:
env = gym.make('FrozenLake-v1', desc=None, map_name="4x4", is_slippery=False)  ##, render_mode = 'human'
action_space_size= env.action_space.n
obs_space_size= env.observation_space.n

q_table= np.zeros((obs_space_size, action_space_size))
print (q_table)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [3]:
num_episodes= 100
t= 1000 ##Steps per episode
lr= 0.1
dr= 0.99
exp_r= 1
max_exp_r= 1
min_exp_r=0.01
exploration_decay_rate= 0.01


In [4]:
##training
total_rewards=[]
episode= 1
while episode<= num_episodes:
    total= 0
    observation, info = env.reset()
    reward_arr=[]
    for steps in range(t):
        exp_r_threshold = random.uniform (0,1)
        if exp_r_threshold > exp_r:
            action=np.argmax(q_table[observation, :])
        else:
            action = env.action_space.sample()
        new_observation, reward, terminated, truncated, info= env.step(action)
        ##print ("New Observation:", new_observation)
        ##print ("Action:", action)
        ##Update q_table
        q_table[observation, action]= (q_table[observation, action]*(1-lr)) + (lr* (reward+ (dr * np.max(q_table[new_observation, :]))))
        observation= new_observation
        total = total+reward
        reward_arr.append(reward)
        if terminated or truncated:
           observation, info= env.reset()
    mean_reward= np.mean(reward_arr)
    ##print (f"Average reward for {episode} episode is: ", mean_reward)
    ##print ("Cumulative Reward:", total)
    ##print ("Episode:", episode)
    exp_r= min_exp_r + (max_exp_r- min_exp_r)* np.exp(-exploration_decay_rate*episode)
    episode=episode+1
    total_rewards.append(total)

print ("Average rewards:", np.mean(total_rewards))
print ("Updated Q table:", q_table)
# Close the environment
env.close()

Average rewards: 38.97
Updated Q table: [[0.94148015 0.95099005 0.95099005 0.94148015]
 [0.94148015 0.         0.96059601 0.95099005]
 [0.95099005 0.970299   0.95099005 0.96059601]
 [0.96059601 0.         0.95099005 0.95099005]
 [0.95099005 0.96059601 0.         0.94148015]
 [0.         0.         0.         0.        ]
 [0.         0.9801     0.         0.96059601]
 [0.         0.         0.         0.        ]
 [0.96059601 0.         0.970299   0.95099005]
 [0.96059601 0.9801     0.9801     0.        ]
 [0.970299   0.99       0.         0.970299  ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.9801     0.99       0.970299  ]
 [0.9801     0.99       1.         0.9801    ]
 [0.         0.         0.         0.        ]]


In [5]:
##Testing
env = gym.make('FrozenLake-v1', desc=None, map_name="4x4", is_slippery=False, render_mode = 'human')  ##
episode=0
while episode<= 4:
    observation, info = env.reset()
    for steps in range(100):
        action=np.argmax(q_table[observation, :])   ##Decides whether at that position, it should go up, down left or right depending on q score
        new_observation, reward, terminated, truncated, info= env.step(action)
        observation= new_observation
        if terminated or truncated:
            if reward==1:
                print ("You have reached the goal!")
            else:
                ##observation, info= env.reset()
                print ("Oops! You are in the hole!")
            break
                
    episode= episode+1

env.close()

You have reached the goal!
You have reached the goal!
You have reached the goal!
You have reached the goal!
You have reached the goal!


In [6]:
q_table

array([[0.94148015, 0.95099005, 0.95099005, 0.94148015],
       [0.94148015, 0.        , 0.96059601, 0.95099005],
       [0.95099005, 0.970299  , 0.95099005, 0.96059601],
       [0.96059601, 0.        , 0.95099005, 0.95099005],
       [0.95099005, 0.96059601, 0.        , 0.94148015],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.9801    , 0.        , 0.96059601],
       [0.        , 0.        , 0.        , 0.        ],
       [0.96059601, 0.        , 0.970299  , 0.95099005],
       [0.96059601, 0.9801    , 0.9801    , 0.        ],
       [0.970299  , 0.99      , 0.        , 0.970299  ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.9801    , 0.99      , 0.970299  ],
       [0.9801    , 0.99      , 1.        , 0.9801    ],
       [0.        , 0.        , 0.        , 0.        ]])

In [7]:
updated_q=np.zeros((16, 4))
for r in range (len(q_table)):
    if q_table[r].any()>0:
        c= np.argmax(q_table[r])
        updated_q[r][c]= 1
    else:
        c= 4
    print (f"Row {r}:", c)
print (updated_q)
        

Row 0: 1
Row 1: 2
Row 2: 1
Row 3: 0
Row 4: 1
Row 5: 4
Row 6: 1
Row 7: 4
Row 8: 2
Row 9: 1
Row 10: 1
Row 11: 4
Row 12: 4
Row 13: 2
Row 14: 2
Row 15: 4
[[0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 0.]]


In [9]:
len(updated_q)

16

In [50]:
##To find the required movements: 0- Move left, 1- Move Down, 2- Move Right, 3- Move Up
move_arr=[]
for r in range (len(updated_q)):
    movement= np.argmax(updated_q[r])
    move_arr.append(movement)
move_arr

[1, 2, 1, 0, 1, 0, 1, 0, 2, 1, 1, 0, 0, 2, 2, 0]

In [31]:
def find_initial_pos(x_grid, y_grid):  ##give dimensions of the map
    x_num= x_grid
    y_num =y_grid-1
    return 0, y_num, x_num
    
def get_coor(x, y, x_num, y_num):  ##find co-ordinates 
    moves =  (x)+((y_num-y)*x_num) 
    return moves 

def find_coor(x,y, ac):  ##update coordinates
    if ac == 0:
        x=x-1
        y=y

    if ac== 1:
        x=x
        y=y-1
        
    if ac== 2:
        x=x+1
        y=y
        
    if ac== 3:
        x=x
        y=y+1
        
    return x, y

def action(move_arr):
    moves=0
    x_grid=4
    y_grid=4
    final_dest= (x_grid*y_grid)-1
    y_num= y_grid-1
    x, y, x_grid = find_initial_pos(x_grid, y_grid)
    tr_arr=[]
    while moves!=final_dest:
        moves= get_coor(x, y, x_grid, y_num)  ##finds the coordinates to search from the observation array
        ac= move_arr[moves]  ##action to be performed
        ##perform action in the arduino robot
        up_x, up_y = find_coor(x, y, ac)   ##updates the x and y values
        x=up_x
        y=up_y
        tr_arr.append(moves)
        print (x,y)
    return tr_arr


In [33]:
req_arr= action(move_arr)

0 2
0 1
1 1
1 0
2 0
3 0
2 0


In [37]:
req_arr

[0, 4, 8, 9, 13, 14, 15]

In [48]:
nec_act=[]
for ind in range (len(req_arr)-1):
    a = req_arr[ind]
    b= move_arr[a]
    nec_act.append(b)
print (nec_act)

[1, 1, 2, 1, 2, 2]


In [49]:
# Generate the content of the header file
header_file_name = 'dataArray.h'
header_content = f"""#ifndef DATA_ARRAY_H
#define DATA_ARRAY_H

const int myArray[] = {{{', '.join(map(str, nec_act))}}};
const int arrayLength = sizeof(myArray) / sizeof(myArray[0]);

#endif // DATA_ARRAY_H
"""
with open(header_file_name, 'w') as header_file:
    header_file.write(header_content)

print(f'{header_file_name} has been generated with the array data.')


dataArray.h has been generated with the array data.
